<a href="https://colab.research.google.com/github/MrHoosha/OpenManus/blob/main/datacollector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# نصب کتابخانه‌های مورد نیاز
!pip install pandas pytz openpyxl xlsxwriter ta numpy matplotlib --quiet

# وارد کردن کتابخانه‌ها
import pandas as pd
import numpy as np
import pytz
from datetime import datetime
from google.colab import files
import matplotlib.pyplot as plt
from ta.trend import MACD
from ta.momentum import RSIIndicator
from ta.volatility import BollingerBands
import xlsxwriter

# -------------------- بخش ۱: خواندن و پیش‌پردازش داده‌ها --------------------
url = "https://www.cryptodatadownload.com/cdd/Binance_BTCUSDT_minute.csv"
try:
    df = pd.read_csv(url, skiprows=1)
except Exception as e:
    print("⚠️ خطا در خواندن داده‌ها! لطفا اتصال اینترنت را بررسی کنید.")
    exit()

# تبدیل تاریخ و فیلتر داده‌ها
df['date'] = pd.to_datetime(df['date'])
df['date_only'] = df['date'].dt.date
target_date = "2021-10-01"
filtered_data = df[df['date_only'] == pd.to_datetime(target_date).date()]

if len(filtered_data) == 0:
    print("⚠️ داده‌ای برای تاریخ انتخابی یافت نشد!")
    exit()
filtered_data = filtered_data.sort_values(by='date')

# -------------------- بخش ۲: محاسبات تکنیکال پیشرفته --------------------
# ۱. میانگین متحرک و RSI
filtered_data['MA20'] = filtered_data['close'].rolling(20).mean().round(2)
filtered_data['RSI'] = RSIIndicator(filtered_data['close'], window=14).rsi().round(2)

# ۲. MACD و باندهای بولینگر
macd = MACD(filtered_data['close'])
filtered_data['MACD'] = macd.macd().round(2)
filtered_data['MACD_signal'] = macd.macd_signal().round(2)
bb = BollingerBands(filtered_data['close'])
filtered_data['BB_upper'] = bb.bollinger_hband().round(2)
filtered_data['BB_lower'] = bb.bollinger_lband().round(2)

# ۳. حجم و نوسانات
filtered_data['Volume_Ratio'] = (filtered_data['Volume USDT'] / filtered_data['Volume USDT'].rolling(20).mean()).round(2)
filtered_data['Volatility'] = (filtered_data['high'] - filtered_data['low']).round(2)

# -------------------- بخش ۳: الگوهای کندلی --------------------
filtered_data['Candle_Body'] = (filtered_data['close'] - filtered_data['open']).abs().round(2)
filtered_data['Candle_Size'] = (filtered_data['high'] - filtered_data['low']).round(2)
filtered_data['Upper_Shadow'] = (filtered_data['high'] - filtered_data[['open','close']].max(axis=1)).round(2)
filtered_data['Lower_Shadow'] = (filtered_data[['open','close']].min(axis=1) - filtered_data['low']).round(2)

# شناسایی الگوها
conditions = [
    (filtered_data['Candle_Body'] == 0),
    (filtered_data['Candle_Body'] < 0.3 * filtered_data['Candle_Size']) & (filtered_data['Lower_Shadow'] > 2 * filtered_data['Upper_Shadow']),
    (filtered_data['Candle_Body'] < 0.3 * filtered_data['Candle_Size']) & (filtered_data['Upper_Shadow'] > 2 * filtered_data['Lower_Shadow'])
]
choices = ['Doji', 'Hammer', 'Shooting Star']
filtered_data['Candle_Pattern'] = np.select(conditions, choices, default='None')

# -------------------- بخش ۴: استراتژی نهنگ‌ها --------------------
# انباشت نهنگ
filtered_data['Whale_Accumulation'] = np.where(
    (filtered_data['Volume_Ratio'] > 1.5) &
    (filtered_data['Volatility'] < 1.0) &
    ((filtered_data['close'] - filtered_data['open']) / filtered_data['open'] * 100 < 0.5),
    '✅ انباشت', ''
)

# توزیع نهنگ
filtered_data['Whale_Distribution'] = np.where(
    (filtered_data['Volume_Ratio'] > 1.5) &
    ((filtered_data['close'] - filtered_data['open']) / filtered_data['open'] * 100 < -0.3) &
    (filtered_data['close'] > filtered_data['MA20']),
    '❌ توزیع', ''
)

# -------------------- بخش ۵: تشخیص وایپ‌اوت --------------------
filtered_data['15m_Change'] = (filtered_data['close'].pct_change(15) * 100).round(2)
filtered_data['15m_Volume'] = filtered_data['Volume USDT'].rolling(15).sum().round(2)
filtered_data['1h_Avg_Volume'] = filtered_data['Volume USDT'].rolling(60).mean().round(2)

wipeout_conditions = [
    (filtered_data['15m_Change'] < -5) & (filtered_data['15m_Volume'] > 2 * filtered_data['1h_Avg_Volume']),
    (filtered_data['15m_Change'].between(-5, -2)) & (filtered_data['15m_Volume'] > 3 * filtered_data['1h_Avg_Volume'])
]
wipeout_labels = ['وایپ‌اوت اصلی', 'وایپ‌اوت جزئی']
filtered_data['Wipeout'] = np.select(wipeout_conditions, wipeout_labels, default='')

# -------------------- بخش ۶: شکاف‌های نقدینگی --------------------
def detect_gaps(df):
    df['Prev_High'] = df['high'].shift(1)
    df['Prev_Low'] = df['low'].shift(1)
    df['Gap_Percent'] = np.where(
        df['open'] > df['Prev_High'],
        ((df['open'] - df['Prev_High']) / df['Prev_High'] * 100).round(2),
        np.where(
            df['open'] < df['Prev_Low'],
            ((df['Prev_Low'] - df['open']) / df['Prev_Low'] * 100).round(2),
            0
        )
    )
    df['Liquidity_Gap'] = np.where(
        df['Gap_Percent'].abs() >= 0.5,
        np.where(df['Gap_Percent'] > 0, 'صعودی 🟢', 'نزولی 🔴'),
        'بدون شکاف'
    )
    return df

filtered_data = detect_gaps(filtered_data)

# -------------------- بخش ۷: نامگذاری فارسی و فرمتبندی --------------------
persian_columns = {
    'date': 'تاریخ',
    'open': 'باز',
    'high': 'بالا',
    'low': 'پایین',
    'close': 'بسته',
    'Volume USDT': 'حجم',
    'MA20': 'میانگین متحرک 20',
    'RSI': 'شاخص قدرت نسبی',
    'MACD': 'مکدی',
    'BB_upper': 'بولینگر بالا',
    'BB_lower': 'بولینگر پایین',
    'Candle_Pattern': 'الگوی کندلی',
    'Whale_Accumulation': 'انباشت نهنگ',
    'Whale_Distribution': 'توزیع نهنگ',
    'Wipeout': 'وایپ‌اوت',
    'Liquidity_Gap': 'شکاف نقدینگی'
}
filtered_data = filtered_data.rename(columns=persian_columns)

# -------------------- بخش ۸: ذخیره در اکسل با فرمتبندی حرفه‌ای --------------------
output_file = f"BTC_Analysis_{target_date}.xlsx"
with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
    filtered_data.to_excel(writer, sheet_name='داده‌ها', index=False)

    # ایجاد نمودارها
    workbook = writer.book
    worksheet = writer.sheets['داده‌ها']

    # نمودار OHLC
    ohlc_chart = workbook.add_chart({'type': 'stock'})
    ohlc_chart.add_series({
        'name': 'قیمت',
        'categories': ['داده‌ها', 1, 0, len(filtered_data), 0],
        'values': ['داده‌ها', 1, 1, len(filtered_data), 4],
        'line': {'color': '#2E75B6'}
    })
    worksheet.insert_chart('M2', ohlc_chart)

    # نمودار شکاف‌ها
    gap_chart = workbook.add_chart({'type': 'column'})
    gap_chart.add_series({
        'name': 'شکاف نقدینگی',
        'categories': ['داده‌ها', 1, 0, len(filtered_data), 0],
        'values': ['داده‌ها', 1, filtered_data.columns.get_loc('شکاف نقدینگی'), len(filtered_data), filtered_data.columns.get_loc('شکاف نقدینگی')],
        'colors': ['#00B050', '#FF0000']
    })
    worksheet.insert_chart('M20', gap_chart)

    # فرمتبندی شرطی
    format_green = workbook.add_format({'bg_color': '#C6EFCE', 'font_color': '#006100'})
    format_red = workbook.add_format({'bg_color': '#FFC7CE', 'font_color': '#9C0006'})

    # RSI
    worksheet.conditional_format('G2:G999', {
        'type': 'cell',
        'criteria': '<=',
        'value': 30,
        'format': format_green
    })
    worksheet.conditional_format('G2:G999', {
        'type': 'cell',
        'criteria': '>=',
        'value': 70,
        'format': format_red
    })

print("✅ فایل اکسل با موفقیت ایجاد شد!")
files.download(output_file)

✅ فایل اکسل با موفقیت ایجاد شد!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>